In [11]:
import os
from river import datasets
from river import ensemble
from confluent_kafka import Producer,Consumer
import certifi
import time
import json
import uuid

In [12]:
#Feature Topic
feature_topic = 'features'

#flush_size and sleep_time throttle the number of feature records written to the feature topic per second
flush_size=800
sleep_time=1

#Maximum number of records processed. We use a small number 5000. But a typical size would be 100000
#In the practice, the features are arriving continuously.
max_size=5000



In [13]:
user= os.environ['kafka_username']
password= os.environ['kafka_password']
bsts= os.environ['kafka_bootstrap_servers']

In [14]:
conf = {'bootstrap.servers': bsts,
            'sasl.mechanism': 'PLAIN',
            'security.protocol': 'SASL_SSL',
            'ssl.ca.location': certifi.where(),
            'sasl.username': user,
            'sasl.password': password,
            'batch.num.messages': 4000,
            'linger.ms': 100,
            'client.id': 'producer-icde-2023'}
producer = Producer(conf)    

In [15]:
#Dataset used
dataset = datasets.MaliciousURL()
data = dataset.take(max_size)
end=0
cnt = 0
st = time.time()
abs_st = time.time()
for f, y in data:
    cnt = cnt + 1    
    d = {}
    d['id'] = str(uuid.uuid4())
    d['f']=f
    d['y']=str(y).lower()
    d['st']=time.time()  
            
    v= json.dumps(d).encode('utf-8')
    try:
        producer.produce(feature_topic, value=v, key=str(cnt))
        if(cnt%flush_size==0):
            print(f'Queue full, flushing {cnt}')
            producer.flush()
            time.sleep(sleep_time)
    except:
      print(f'Queue full, flushing {cnt}')
      producer.flush()
      end = time.time()
      print(f'flushing count - {cnt}, time taken in seconds- {end-st} ')        
        
      producer.produce(feature_topic, value=v, key=str(cnt))
 
producer.flush()
end = time.time()
print(f'final flushing count - {cnt}, time taken in seconds- {end-abs_st} ')        

Queue full, flushing 800
Queue full, flushing 1600
Queue full, flushing 2400
Queue full, flushing 3200
Queue full, flushing 4000
Queue full, flushing 4800
final flushing count - 5000, time taken in seconds- 8.400670051574707 
